<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [16]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [17]:
if IN_COLAB:
    !pip install openai
    !pip install requests

if IN_COLAB:
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [18]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    !git clone "https://github.com/samgregson/GHPT-experiments"
    !pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

# OpenAI

In [19]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

In [28]:
from langsmith import traceable

# create the openai api call function
@traceable
def call_openai(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    if system_prompt == "":
        messages=[
            {"role": "user", "content": prompt}
        ]
    else:
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]

    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages
    )

    return completion

test connection

In [30]:
model: str = "gpt-3.5-turbo-1106"
temperature: float = 0

prompt="say this is a test"
system_prompt=""

completion = call_openai(prompt=prompt, system_prompt=system_prompt, model=model, temperature=temperature)
result = completion.choices[0].message.content
print(result)

Failed to get info from https://api.smith.langchain.com: SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /info (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))"))


This is a test.


Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)\')))"))')
Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)\')

# GHPT

### Import prompts

In [22]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

In [23]:
# example use:
prompt_template.format(QUESTION="create a sphere")

'\n// Question : How do I add two numbers inside of Grasshopper?\n// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together\n// JSON:\n{\n\t"Advice": "Make sure to set the number sliders to the correct value",\n\t"Additions": [\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 1,\n\t\t\t"value": "0..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 2,\n\t\t\t"value": "-50..25..100"\n\t\t},\n\t\t{\n\t\t\t"Name": "Addition",\n\t\t\t"Id": 3\n\t\t}\n\t],\n\t"Connections": [\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "A"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 1,\n\t\t\t\t"ParameterName": "number"\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"To": {\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "B"\n\t\t\t},\n\t\t\t"From": {\n\t\t\t\t"Id": 2,\n\t\t\t\t"Parame

In [24]:
system_prompt

"\nYou are a Grasshopper Expert and are going to help create Grasshopper Definitions.\nKeep the answers short and concise.\nMake sure you create and connect a component for every non-optional input\nIf you're not sure about the answer, but think there's additional information that could help you, please ask for that information.\n\nAlways use the given format, avoid any devitation.\n"

## Baseline Call

In [25]:
# model: str = "gpt-3.5-turbo-1106"
model: str = "gpt-3.5-turbo"
# model = "gpt-4"
temperature: float = 0

prompt=system_prompt + prompt_template.format(QUESTION="sphere")

completion = call_openai(prompt=prompt, system_prompt="", model=model, temperature=temperature)
result = completion.choices[0].message
print(result.content)

I'm sorry, I don't have enough information to provide a Grasshopper definition for a sphere. Could you please provide more details or context?
